In [1]:
import cv2
import pandas as pd
import math
# Load the CSV file
file_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\excel_examples\Full_length.xlsx"
data=pd.read_excel(file_path)
# Process each row in the CSV


# Load the image
image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1/full body\undistorted_GX010067_33_625.jpg_gamma.jpg" # Adjust the path as needed
image = cv2.imread(image_path)

for index, row in data.iterrows():
    # Extract the details from the row
    label = row['Label']

    bx = int(row['BX']*5.3)
    by = int(row['BY']*5.3)
    width = int(row['Width']*5.3)
    height = int(row['Height']*5.3)
    length = int(row['Length'])
    angle = row['Angle']


    #draw rectangle
    color = (0, 255, 0)  # Green color in BGR
    thickness = 2
    image = cv2.rectangle(image, (bx, by), (bx + width, by + height), color, thickness)




    print(f"Processing row {index}: {label}, bx={bx}, by={by}, width={width}, height={height}, length={length}")   

    if angle<0 and angle>-90:
            start_point = (bx, by+height)
            end_point = ( bx+width,by+height )             
    elif angle < -90 and angle > -180:
             start_point = (bx, by+height)
             end_point = ( bx+width,by  )  

    elif angle < 180 and angle > 90:
             start_point = (bx, by)
             end_point = ( bx+width,by+height  )
    else:
                start_point = (bx, by+height)
                end_point = ( bx+width,by+height ) 
          



    color = (255, 0, 0)  # Blue color in BGR
    thickness = 2
    image = cv2.line(image, start_point, end_point, color, thickness)

    # Draw a large circle at (bx, by)
    circle_radius = 10
    circle_color = (0, 0, 255)  # Red color in BGR
    image = cv2.circle(image, (bx, by), circle_radius, circle_color, -1)

    # Draw circles at the corners of the bounding box
    corners = [(bx + width, by), (bx, by + height), (bx + width, by + height)]
    for corner in corners:
        image = cv2.circle(image, corner, circle_radius, circle_color, -1)

    # Add the bx, by text
    text_bx_by = f'({bx}, {by})'
    org_bx_by = (bx + 15, by + 15)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    text_color = (0, 255, 0)  # Green color in BGR
    text_thickness = 2
    image = cv2.putText(image, text_bx_by, org_bx_by, font, font_scale, text_color, text_thickness, cv2.LINE_AA)

    # Add the length text
    text_length = f'Length: {length:.2f}'
    org_length = (bx+width, by - 10)
    image = cv2.putText(image, text_length, org_length, font, font_scale, text_color, text_thickness, cv2.LINE_AA)


    # Save or display the image
output_path = f'annot.jpg'  # Adjust the path as needed
cv2.imwrite(output_path, image)
print(f"Saved output image to {output_path}")






Processing row 0: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=2875, by=1822, width=51, height=17, length=10
Processing row 1: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=2875, by=1822, width=51, height=17, length=53
Processing row 2: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=2878, by=524, width=489, height=462, length=126
Processing row 3: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=3643, by=1762, width=344, height=383, length=96
Processing row 4: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=997, by=1944, width=821, height=225, length=160
Processing row 5: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=82, by=2094, width=505, height=44, length=95
Saved output image to annot.jpg


In [2]:
import cv2
import pandas as pd
import math
# Load the CSV file
file_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\each video from research extracted to images\kalkar - january 2024/1.1/65\Results.csv"
data=pd.read_csv(file_path)
# Process each row in the CSV


# Load the image
image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\each video from research extracted to images\kalkar - january 2024/1.1/65\GX010082_212_2827.jpg"
image = cv2.imread(image_path)

for index, row in data.iterrows():
    # Extract the details from the row
    label = row['Label']

    bx = int(row['BX'])
    by = int(row['BY'])
    width = int(row['Width'])
    height = int(row['Height'])
    length = int(row['Length'])
    angle = row['Angle']



    print(f"Processing row {index}: {label}, bx={bx}, by={by}, width={width}, height={height}, length={length}")   

    if angle>=0 and angle<45:
            start_point =     (bx, by + height)
            end_point =    (bx + width, by)  
    elif angle >= 45 and angle <=90:
            start_point =    (bx, by + height)
            end_point =  (bx + width, by)       
    elif angle > 90 and angle <=135:
             start_point = (bx, by)
             end_point = ( bx+width,by+height )

    elif angle >135 and angle <=180:
                start_point = (bx, by)
                end_point = ( bx+width,by+height )           

    elif angle >-45  and angle < 0:
             start_point = (bx, by)
             end_point = ( bx+width,by+height  )

    elif angle >-90 and angle < -45:
            start_point = (bx, by)
            end_point = ( bx+width,by+height  )             
    elif angle >= -135 and angle < -90:
             start_point =(bx, by + height)
             end_point =  (bx + width, by)        

    else:
            start_point =  (bx, by + height)
            end_point = (bx + width, by)
          

    image = cv2.rectangle(image, (bx, by), (bx + width, by + height), color, thickness)


    color = (255, 0, 0)  # Blue color in BGR
    thickness = 2
    image = cv2.line(image, start_point, end_point, color, thickness)

    # Draw a large circle at (bx, by)
    circle_radius = 10
    circle_color = (0, 0, 255)  # Red color in BGR
    image = cv2.circle(image, (bx, by), circle_radius, circle_color, -1)

    # Draw circles at the corners of the bounding box
    corners = [
        (bx, by),                      # Top-left
        (bx + width, by),              # Top-right
        (bx, by + height),             # Bottom-left
        (bx + width, by + height)      # Bottom-right
        ]

# Define colors for each corner (BGR format)
    colors = [
(255, 0, 0),   # Blue for Top-left
(0, 255, 0),   # Green for Top-right
(0, 0, 255),   # Red for Bottom-left
(0, 255, 255)  # Yellow for Bottom-right
]

# Circle radius
    circle_radius = 10

# Draw circles at each corner with different colors
    for corner, color in zip(corners, colors):
        image = cv2.circle(image, corner, circle_radius, color, -1)

    # Add the bx, by text
text_bx_by = f'({bx}, {by})'
org_bx_by = (bx + 15, by + 15)
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
text_color = (0, 255, 0)  # Green color in BGR
text_thickness = 2
image = cv2.putText(image, text_bx_by, org_bx_by, font, font_scale, text_color, text_thickness, cv2.LINE_AA)

# Add the length text
text_length = f'Length: {length:.2f}'
org_length = (bx+width, by - 10)
image = cv2.putText(image, text_length, org_length, font, font_scale, text_color, text_thickness, cv2.LINE_AA)


    # Save or display the image
output_path = f'annot2.jpg'  # Adjust the path as needed
cv2.imwrite(output_path, image)
print(f"Saved output image to {output_path}")






Processing row 0: GX010082_212_2827.jpg, bx=3004, by=736, width=5, height=441, length=440
Processing row 1: GX010082_212_2827.jpg, bx=3276, by=668, width=5, height=493, length=492
Processing row 2: GX010082_212_2827.jpg, bx=3456, by=864, width=537, height=21, length=536
Processing row 3: GX010082_212_2827.jpg, bx=3448, by=980, width=589, height=17, length=588
Processing row 4: GX010082_212_2827.jpg, bx=3888, by=524, width=225, height=225, length=316
Processing row 5: GX010082_212_2827.jpg, bx=3788, by=476, width=205, height=205, length=288
Processing row 6: GX010082_212_2827.jpg, bx=2476, by=360, width=401, height=369, length=543
Processing row 7: GX010082_212_2827.jpg, bx=2424, by=440, width=357, height=329, length=484
Processing row 8: GX010082_212_2827.jpg, bx=3560, by=1238, width=200, height=58, length=207
Processing row 9: GX010082_212_2827.jpg, bx=3531, by=1193, width=197, height=63, length=205
Processing row 10: GX010082_212_2827.jpg, bx=2818, by=1242, width=160, height=42, leng

In [ ]:
import pandas as pd
import numpy as np

# Load the three Excel files into DataFrames
file_1_path = "path_to_first_excel_file.xlsx"
file_2_path = "path_to_second_excel_file.xlsx"
file_3_path = "path_to_third_excel_file.xlsx"

data_1 = pd.read_excel(file_1_path)
data_2 = pd.read_excel(file_2_path)
data_3 = pd.read_excel(file_3_path)

# Add a 'File' column to each DataFrame to identify the source file
data_1['File'] = 'File 1'
data_2['File'] = 'File 2'
data_3['File'] = 'File 3'

# Combine all DataFrames into one
combined_data = pd.concat([data_1, data_2, data_3])

# Separate the scale rows (first two rows) from prawn data
scales_data = combined_data.groupby('Image_Name').apply(lambda x: x.iloc[:2]).reset_index(drop=True)
prawn_data = combined_data.groupby('Image_Name').apply(lambda x: x.iloc[2:]).reset_index(drop=True)

# Define a tolerance for bounding box area comparison in the image (percentage tolerance)
area_tolerance = 0.1  # 10% tolerance

# Calculate the area of the bounding box in the image
def calculate_area(row):
    return row['Width'] * row['Height']

# Function to check if two bounding box areas in the image are within the tolerance
def areas_are_within_tolerance(area1, area2, tolerance):
    return abs(area1 - area2) <= (tolerance * area1)

# Create a unique identifier for each prawn based on the bounding box area in the image with tolerance
def create_prawn_id(row, tolerance, prawn_data):
    area1 = calculate_area(row)
    bx1, by1 = row['BX'], row['BY']
    for i, r in prawn_data.iterrows():
        if r['Prawn_ID'] is not None:
            continue
        area2 = calculate_area(r)
        bx2, by2 = r['BX'], r['BY']
        if areas_are_within_tolerance(area1, area2, tolerance) and (bx1 == bx2 and by1 == by2):
            prawn_data.at[i, 'Prawn_ID'] = f"{row['Image_Name']}_{i}"
            return f"{row['Image_Name']}_{i}"
    return f"{row['Image_Name']}_{row.name}"

# Initialize Prawn_ID with None
prawn_data['Prawn_ID'] = None

# Apply the function to create a unique ID for each prawn based on the bounding box area in the image with tolerance
prawn_data['Prawn_ID'] = prawn_data.apply(lambda row: create_prawn_id(row, area_tolerance, prawn_data), axis=1)

# Group by the unique identifier (Prawn_ID) to combine measurements of the same prawn within the same image
grouped = prawn_data.groupby('Prawn_ID')

# Calculate statistics for each group (each prawn within each image)
statistics_per_prawn = grouped['Length'].agg(
    Avg_Length=np.mean,
    Std_Dev=np.std,
    Median_Length=np.median,
    Measurement_Count='count'
)

# Calculate the uncertainty (standard deviation of the mean)
statistics_per_prawn['Uncertainty'] = statistics_per_prawn['Std_Dev'] / np.sqrt(statistics_per_prawn['Measurement_Count'])

# Filter out groups with fewer than 3 measurements (optional)
statistics_per_prawn = statistics_per_prawn[statistics_per_prawn['Measurement_Count'] >= 3]

# Calculate overall statistics across all prawns in all images
overall_statistics = {
    "Overall Avg Length": np.mean(statistics_per_prawn['Avg_Length']),
    "Overall Std Dev": np.std(statistics_per_prawn['Avg_Length']),
    "Overall Median Length": np.median(statistics_per_prawn['Avg_Length']),
    "Overall Uncertainty": np.std(statistics_per_prawn['Avg_Length']) / np.sqrt(len(statistics_per_prawn))
}

# Display the scale statistics
print("Scale Statistics:")
print(scales_data.groupby('Image_Name')['Length'].agg(
    Avg_Scale_Length=np.mean,
    Std_Scale_Length=np.std,
    Median_Scale_Length=np.median
))

# Display the statistics for each prawn within each image
print("\nPrawn Length Statistics:")
print(statistics_per_prawn)

# Display overall statistics
print("\nOverall Statistics:")
for key, value in overall_statistics.items():
    print(f"{key}: {value:.2f}")

# Optionally, save the statistics to a new Excel file
statistics_per_prawn.to_excel("prawn_length_statistics_per_image.xlsx")


# looks good

In [8]:
import pandas as pd

# Load the three Excel files into DataFrames
file_1_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\compile 3 files/1_Carapace.xlsx"
file_2_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\compile 3 files/2_Carapace.xlsx"
file_3_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\compile 3 files/3_Carapace.xlsx"

data_1 = pd.read_excel(file_1_path)
data_2 = pd.read_excel(file_2_path)
data_3 = pd.read_excel(file_3_path)

# Function to extract the pixel-to-mm scale from the second row of each image
def extract_scale(data):
    return data.groupby('Label').apply(lambda x: x.iloc[1]['Length'] / 10).reset_index(name='Scale')

# Extract scales for each dataset
scale_1 = extract_scale(data_1)
scale_2 = extract_scale(data_2)
scale_3 = extract_scale(data_3)

# Function to adjust bounding box coordinates based on the pixel-to-mm scale
def adjust_bounding_box(data, scale):
    data = data.copy()
    prawn_data = data.groupby('Label').apply(lambda x: x.iloc[2:]).reset_index(drop=True)  # Skip the first two rows
    prawn_data['BX'] *= scale
    prawn_data['BY'] *= scale
    prawn_data['Width'] *= scale
    prawn_data['Height'] *= scale
    return prawn_data

# Adjust bounding boxes in each dataset for each image separately
def process_image(image_label, data_1, data_2, data_3, scale_1, scale_2, scale_3, tolerance):
    img_data_1 = data_1[data_1['Label'] == image_label]
    img_data_2 = data_2[data_2['Label'] == image_label]
    img_data_3 = data_3[data_3['Label'] == image_label]

    scale_val_1 = scale_1[scale_1['Label'] == image_label]['Scale'].values[0]
    scale_val_2 = scale_2[scale_2['Label'] == image_label]['Scale'].values[0]
    scale_val_3 = scale_3[scale_3['Label'] == image_label]['Scale'].values[0]

    prawn_data_1 = adjust_bounding_box(img_data_1, scale_val_1)
    prawn_data_2 = adjust_bounding_box(img_data_2, scale_val_2)
    prawn_data_3 = adjust_bounding_box(img_data_3, scale_val_3)

    # Assign PrawnID by comparing bounding boxes with tolerance within the same image
    prawn_data_1['PrawnID'] = None
    prawn_data_2['PrawnID'] = None
    prawn_data_3['PrawnID'] = None

    for i, row1 in prawn_data_1.iterrows():
        bx1, by1, width1, height1 = row1['BX'], row1['BY'], row1['Width'], row1['Height']
        area1 = width1 * height1

        for j, row2 in prawn_data_2.iterrows():
            bx2, by2, width2, height2 = row2['BX'], row2['BY'], row2['Width'], row2['Height']
            area2 = width2 * height2

            if abs(area1 - area2) <= (tolerance * area1) and abs(bx1 - bx2) <= tolerance and abs(by1 - by2) <= tolerance:
                prawn_data_1.at[i, 'PrawnID'] = f"{image_label}_{i}"
                prawn_data_2.at[j, 'PrawnID'] = f"{image_label}_{i}"
                break

        for k, row3 in prawn_data_3.iterrows():
            bx3, by3, width3, height3 = row3['BX'], row3['BY'], row3['Width'], row3['Height']
            area3 = width3 * height3

            if abs(area1 - area3) <= (tolerance * area1) and abs(bx1 - bx3) <= tolerance and abs(by1 - by3) <= tolerance:
                prawn_data_1.at[i, 'PrawnID'] = f"{image_label}_{i}"
                prawn_data_3.at[k, 'PrawnID'] = f"{image_label}_{i}"
                break

    return prawn_data_1, prawn_data_2, prawn_data_3

# Set tolerance for bounding box comparison
tolerance = 15 # Adjust based on acceptable variance

# Process each image separately
processed_data_1 = []
processed_data_2 = []
processed_data_3 = []

for image_label in data_1['Label'].unique():
    img_data_1, img_data_2, img_data_3 = process_image(image_label, data_1, data_2, data_3, scale_1, scale_2, scale_3, tolerance)
    processed_data_1.append(img_data_1)
    processed_data_2.append(img_data_2)
    processed_data_3.append(img_data_3)

# Combine the processed data into DataFrames
final_data_1 = pd.concat(processed_data_1, ignore_index=True)
final_data_2 = pd.concat(processed_data_2, ignore_index=True)
final_data_3 = pd.concat(processed_data_3, ignore_index=True)

# Merge datasets on PrawnID and Label to combine the measurements across the three files
# Merge datasets on PrawnID and Label to combine the measurements across the three files
merged_data = final_data_1.merge(final_data_2, on=['Label', 'PrawnID'], suffixes=('_1', '_2'))

# Rename the columns in final_data_3 to ensure they have a unique suffix
final_data_3 = final_data_3.rename(columns={
    'SN': 'SN_3',
    'Area': 'Area_3',
    'Mean': 'Mean_3',
    'Min': 'Min_3',
    'Max': 'Max_3',
    'BX': 'BX_3',
    'BY': 'BY_3',
    'Width': 'Width_3',
    'Height': 'Height_3',
    'Angle': 'Angle_3',
    'Slice': 'Slice_3',
    'Length': 'Length_3'
})


merged_data = merged_data.merge(final_data_3, on=['Label', 'PrawnID'])
# Check if the length columns are present
print(merged_data.columns)


# Calculate statistics for each prawn
def calculate_statistics(merged_data):
    # Calculate mean, standard deviation, and uncertainty
    merged_data['Mean_Length'] = merged_data[['Length_1', 'Length_2', 'Length_3']].mean(axis=1)
    merged_data['Std_Dev_Length'] = merged_data[['Length_1', 'Length_2', 'Length_3']].std(axis=1)
    merged_data['Uncertainty'] = merged_data['Std_Dev_Length'] / (3 ** 0.5)
    return merged_data

# Apply the statistics calculation
final_data = calculate_statistics(merged_data)

# Concat the bounding box and angle information for verification
final_data['BoundingBox_1'] = final_data[['BX_1', 'BY_1', 'Width_1', 'Height_1']].apply(tuple, axis=1)
final_data['BoundingBox_2'] = final_data[['BX_2', 'BY_2', 'Width_2', 'Height_2']].apply(tuple, axis=1)
final_data['BoundingBox_3'] = final_data[['BX_3', 'BY_3', 'Width_3', 'Height_3']].apply(tuple, axis=1)

final_data['Angle_1'] = final_data['Angle_1']
final_data['Angle_2'] = final_data['Angle_2']
final_data['Angle_3'] = final_data['Angle_3']

# Display the final dataframe with PrawnID, calculated statistics, bounding boxes, and angles
final_data[['Label', 'PrawnID', 'Mean_Length', 'Std_Dev_Length', 'Uncertainty', 
            'BoundingBox_1', 'BoundingBox_2', 'BoundingBox_3', 
            'Angle_1', 'Angle_2', 'Angle_3']]
#save the final data to a new Excel file
final_data.to_excel("final_data.xlsx", index=False)
# Display the final dataframe with PrawnID, calculated statistics, bounding boxes, and angles

Index(['SN_1', 'Label', 'Area_1', 'Mean_1', 'Min_1', 'Max_1', 'BX_1', 'BY_1',
       'Width_1', 'Height_1', 'Angle_1', 'Slice_1', 'Length_1', 'PrawnID',
       'SN_2', 'Area_2', 'Mean_2', 'Min_2', 'Max_2', 'BX_2', 'BY_2', 'Width_2',
       'Height_2', 'Angle_2', 'Slice_2', 'Length_2', 'SN_3', 'Area_3',
       'Mean_3', 'Min_3', 'Max_3', 'BX_3', 'BY_3', 'Width_3', 'Height_3',
       'Angle_3', 'Slice_3', 'Length_3'],
      dtype='object')


# inspect visually

In [1]:
import pandas as pd
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import tempfile

# Load the three Excel files into DataFrames
file_1_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\compile 3 files/1_Carapace.xlsx"
file_2_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\compile 3 files/2_Carapace.xlsx"
file_3_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\compile 3 files/3_Carapace.xlsx"

data_1 = pd.read_excel(file_1_path)
data_2 = pd.read_excel(file_2_path)
data_3 = pd.read_excel(file_3_path)

# Function to extract the pixel-to-mm scale from the second row of each image
def extract_scale(data):
    return data.groupby('Label').apply(lambda x: x.iloc[1]['Length'] / 10).reset_index(name='Scale')

# Extract scales for each dataset
scale_1 = extract_scale(data_1)
scale_2 = extract_scale(data_2)
scale_3 = extract_scale(data_3)

# Function to adjust bounding box coordinates based on the pixel-to-mm scale
def adjust_bounding_box(data, scale):
    data = data.copy()
    data['BX'] *= scale
    data['BY'] *= scale
    data['Width'] *= scale
    data['Height'] *= scale
    return data

# Function to draw bounding boxes and lines
def draw_bounding_boxes_and_lines(image, prawn_data, color, thickness=2):
    bx = int(prawn_data['BX'])
    by = int(prawn_data['BY'])
    width = int(prawn_data['Width'])
    height = int(prawn_data['Height'])
    angle = prawn_data['Angle']

    if angle >= 0 and angle < 45 or angle >= 135 and angle <= 180:
        start_point = (bx, by + height)
        end_point = (bx + width, by)
    else:
        start_point = (bx, by)
        end_point = (bx + width, by + height)

    # Draw the rectangle (bounding box)
    image = cv2.rectangle(image, (bx, by), (bx + width, by + height), color, thickness)
    
    # Draw the line
    image = cv2.line(image, start_point, end_point, color, thickness)

    return image

# Function to find the closest matching bounding box based on location
def find_closest_bbox(bbox, prawn_data):
    """Finds the closest matching bounding box in prawn_data to the given bbox."""
    bx1, by1, width1, height1 = bbox

    def distance(row):
        bx2, by2, width2, height2 = row['BX'], row['BY'], row['Width'], row['Height']
        return np.sqrt((bx1 - bx2)**2 + (by1 - by2)**2)

    closest_row = prawn_data.iloc[prawn_data.apply(distance, axis=1).argmin()]
    return closest_row

# Function to manually assign PrawnIDs by visual inspection
def assign_prawn_ids_manually(image_label, data_1, data_2, data_3, scale_1, scale_2, scale_3):
    img_data_1 = data_1[data_1['Label'] == image_label]
    img_data_2 = data_2[data_2['Label'] == image_label]
    img_data_3 = data_3[data_3['Label'] == image_label]

    scale_val_1 = scale_1[scale_1['Label'] == image_label]['Scale'].values[0]
    scale_val_2 = scale_2[scale_2['Label'] == image_label]['Scale'].values[0]
    scale_val_3 = scale_3[scale_3['Label'] == image_label]['Scale'].values[0]

    # Adjust the bounding boxes according to the scale
    img_data_1 = adjust_bounding_box(img_data_1, scale_val_1)
    img_data_2 = adjust_bounding_box(img_data_2, scale_val_2)
    img_data_3 = adjust_bounding_box(img_data_3, scale_val_3)

    prawn_ids = []
    prawn_id_counter =0

    for i, row1 in img_data_1.iloc[2:].iterrows():
        print(f"Assigning ID for prawn {prawn_id_counter+1} in image {image_label}")


        file_name = image_label.split(":")[1] if ":" in image_label else image_label

        file_root, file_ext = os.path.splitext(file_name)
      
    # Append a .jpg extension if it’s missing
      

        file_name = f"{file_root}{file_ext}.jpg"

       # Load the image (assuming you have a way to load it based on the label)
        image_path = f"C:/Users/gbo10/OneDrive/measurement_paper_images/images used for imageJ/check/stabilized/shai/measurements/1/carapace/{file_name}"  
        
        print(image_path)   
        image = cv2.imread(image_path)

        # Draw bounding boxes and lines from dataset 1
        image = draw_bounding_boxes_and_lines(image, row1, (255, 0, 0))  # Blue for file 1
        
        # Find the closest matching bounding boxes in the other datasets
        closest_bbox_2 = find_closest_bbox((row1['BX'], row1['BY'], row1['Width'], row1['Height']), img_data_2)
        closest_bbox_3 = find_closest_bbox((row1['BX'], row1['BY'], row1['Width'], row1['Height']), img_data_3)

        # Draw bounding boxes and lines from dataset 2 and 3
        image = draw_bounding_boxes_and_lines(image, closest_bbox_2, (0, 255, 0))  # Green for file 2
        image = draw_bounding_boxes_and_lines(image, closest_bbox_3, (0, 0, 255))  # Red for file 3

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Display the image using Matplotlib
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        temp_file_path = temp_file.name
        cv2.imwrite(temp_file_path, image)
        temp_file.close()

        # Open the image using the default image viewer
        os.system(f'start {temp_file_path}')

        # Manually input the ID
        prawn_id = input(f"Enter PrawnID for prawn {prawn_id_counter+1} in image {image_label}: ")
        prawn_ids.append(prawn_id)

        # Delete the temporary file
        os.remove(temp_file_path)


        prawn_id_counter += 1
     

        # Assign PrawnID to closest matches in all datasets
        img_data_1.at[i, 'PrawnID'] = prawn_id
        img_data_2.at[closest_bbox_2.name, 'PrawnID'] = prawn_id
        img_data_3.at[closest_bbox_3.name, 'PrawnID'] = prawn_id

    return img_data_1, img_data_2, img_data_3

# Process each image and assign PrawnIDs manually
processed_data_1 = []
processed_data_2 = []
processed_data_3 = []

for image_label in data_1['Label'].unique():
    img_data_1, img_data_2, img_data_3 = assign_prawn_ids_manually(image_label, data_1, data_2, data_3, scale_1, scale_2, scale_3)
    processed_data_1.append(img_data_1)
    processed_data_2.append(img_data_2)
    processed_data_3.append(img_data_3)

# Combine the processed data into DataFrames
final_data_1 = pd.concat(processed_data_1, ignore_index=True)
final_data_2 = pd.concat(processed_data_2, ignore_index=True)
final_data_3 = pd.concat(processed_data_3, ignore_index=True)

# Save the final data with manually assigned PrawnIDs
final_data_1.to_excel("final_data_1_with_prawn_ids.xlsx", index=False)
final_data_2.to_excel("final_data_2_with_prawn_ids.xlsx", index=False)
final_data_3.to_excel("final_data_3_with_prawn_ids.xlsx", index=False)


Assigning ID for prawn 1 in image carapace:undistorted_GX010067_33_625.jpg_gamma
C:/Users/gbo10/OneDrive/measurement_paper_images/images used for imageJ/check/stabilized/shai/measurements/1/carapace/undistorted_GX010067_33_625.jpg_gamma.jpg
